# Diffusion model fine turning with lora 

✅ 네이버 웹툰 중 특정 웹툰의 얼굴 데이터셋을 학습시켜보자 
<br />
<br />
🔗 lora github : https://github.com/cloneofsimo/lora
<br />
🔗 dataset github : https://github.com/bryandlee/naver-webtoon-data

## 1. 환경 설정

※ 우렁이 기준

In [ ]:
% conda create -n lora python=3.8
% conda activate lora
% git clone https://github.com/cloneofsimo/lora.git && sed -i 's/functools.cache/functools.lru_cache(maxsize=None)/g' ./lora/lora_diffusion/xformers_utils.py && pip install ./lora
% pip install accelerate bitsandbytes-cuda114

## 2. 데이터셋 생성 

✅ title id는 [dataset github](https://github.com/bryandlee/naver-webtoon-data) 참고 
<br />
``` python download_faces.py --titles [title ids to download] ```

In [ ]:
% git clone https://github.com/bryandlee/naver-webtoon-data.git
% python ./naver-webtoon-data/download_faces.py --titles 103759 

## 3. 학습하기
🚨 스타일 학습 스크립트 수정 
<br />
_./lora/training_scripts/multivector_example.sh_
<br />
```
export MODEL_NAME="runwayml/stable-diffusion-v1-5"
export INSTANCE_DIR="./naver-webtoon-data/faces/103759"
export OUTPUT_DIR="./exps/imalryon"
...
--placeholder_tokens="imalryon" 
```

In [ ]:
% sh ./lora/training_scripts/multivector_example.sh

## 4. infrence 하기 

### 4-1. prompt 기반 이미지 생성 (text2img)
1️⃣ 기존 diffusion model로 생성하기 

2️⃣ fine turning한 diffusion model으로 생성하기
<br />
<br />
🚨 `3. 학습하기`의 `OUTPUT_DIR` 경로로 checkpoint 경로 수정 및 `placeholder_tokens` 설정 값으로 prompt 내 `style of <s1><s2>` 부분 수정 

In [ ]:
from lora.lora_diffusion import tune_lora_scale, patch_pipe

# checkpoint 경로 수정 
patch_pipe(
    pipe,
    "./exps/imalryon",
    patch_text=True,
    patch_ti=True,
    patch_unet=True,
)

# 입맛에 맞게 scale 조정 
tune_lora_scale(pipe.unet, 0.20)
tune_lora_scale(pipe.text_encoder, 1.00)

prompt = "style of imalryon, hello kitty face"

torch.manual_seed(0)
image = pipe(prompt, num_inference_steps=50, guidance_scale=7).images[0]
image.save("./inference/lora-test.jpg")
image

### 4-2 img를 가이드라인으로 주면서 생성하기 (img&prt2img)

1️⃣ 기존 diffusion model로 생성하기 
<br />
<br />
✅ [프롬포트 제네레이터](https://huggingface.co/spaces/doevent/prompt-generator) 이용하면 편해요!

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch
from PIL import Image

# 가이드라인으로 주고 싶은 원하는 이미지 경로 설정 
init_image = Image.open("./lora/contents/lora_just_unet.jpg").convert("RGB").resize((512, 512))

model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to(
    "cuda"
)

# 프롬포트 설정
prompt = "style of <s1><s2>, a woman with long, straight hair and bangs, detailed "
negative_prompt = 'deformed, ugly, mutilated, disfigured, text, extra limbs, face cut, head cut, extra fingers, extra arms, mutation, bad proportions, cropped head, malformed limbs, mutated hands, fused fingers, long neck'

torch.manual_seed(1)
image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=init_image, strength=0.75, guidance_scale=7.5).images[0]

image

2️⃣ fine turning한 diffusion model으로 생성하기 

In [ ]:
from lora.lora_diffusion import patch_pipe, tune_lora_scale

patch_pipe(pipe, "./exps/imalryon/final_lora.safetensors", patch_text=True, patch_unet=True, patch_ti=True)

# 입맛에 맞게 scale 조정
tune_lora_scale(pipe.unet, 0.0)
tune_lora_scale(pipe.text_encoder, 0.0)

prompt = "style of imalryon, a woman with long, straight hair and bangs, detailed"
negative_prompt = 'deformed, ugly, mutilated, disfigured, text, extra limbs, face cut, head cut, extra fingers, extra arms, mutation, bad proportions, cropped head, malformed limbs, mutated hands, fused fingers, long neck'

torch.manual_seed(1)
image = pipe(prompt=prompt, negative_prompt=negative_prompt, image=init_image, strength=0.75, guidance_scale=7.5).images[0]

image